In [1]:
# Importing relevant packages
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import pandas.util.testing
import time
from dask import dataframe as dd
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Store the dataset columns as original and performance data 
orig_colnames=['Credit_score','First_pay_dt', 'First_time_HB_Flag', 'Maturity_dt', 'MSA', 'Mortgage_Ins%',
'No_of_units', 'Occupancy_status', 'Orig_CLTV', 'Orig_DTI_ratio', 'Orig_UPB',
'Orig_LTV', 'Orig_Interest_rate', 'Channel', 'PPM_flag', 'Prod_type', 'Property_state',
'Property_type', 'Postal_code', 'Loan_seq_no', 'loan_purpose', 'Orig_loan_term',
'No_of_borrowers', 'Seller_name', 'Service_name', 'Super_conf_flag',
'Pre_harp_loan_seq_no', 'Prog_indicator', 'Harp_indicator', 'Prop_val_method','Interest_only_Indicator' ]

svcg_colnames=['Loan_seq_no', 'Monthly_report_period', 'Curr_act_UPB', 'Curr_loan_delinq_status', 'Loan_age',
'Remain_month_legal_maturity', 'Repurchase_flag', 'Modify_flag', 'Zero_bal_code', 'Zero_bal_effective_dt',
'Curr_interest_rate', 'Curr_def_UPB', 'Due_dt_of_LPI', 'MI_recov','Net_sales_proceeds', 'Non_MI_recov',
'Expenses', 'Legal_costs', 'Maintenance_preserv_costs', 'Tax_&_Ins', 'Misc_Exp',
'Act_loss_cal', 'Modify_cost', 'Step_modify_flag', 'Def_payment_plan', 'Est_LTV', 'Zero_bal_rem_UPB',
'Delinq_acc_interest', 'Delinq_due_disaster', 'Borrower_ass_status_code','Current_month_Modification_cost','Interest_Bearing_UPB' ]


31 32


In [3]:
%%time
#Reading using Dask
# Loop to read the YOY data for both Origination & Performance & appending into a master table

master_df = pd.DataFrame()
for i in range(2015,2018):
    year=''.join(map(str, [i]))
    orig_file_name="\sample_orig_"+year
    globals()["orig_df_"+year] = dd.read_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData"+ orig_file_name + ".txt",delimiter = "|",names= orig_colnames,header = None,dtype={'Loan_age': 'float64','Curr_act_UPB': 'object','Delinq_due_disaster': 'object','Repurchase_flag': 'object','Step_modify_flag': 'object','Est_LTV': 'object','Def_payment_plan': 'object','Modify_cost':'object','Act_loss_cal': 'object','Delinq_acc_interest': 'object','Monthly_report_period': 'object','Zero_bal_rem_UPB': 'object','Service_name': 'object','Remain_month_legal_maturity': 'object','Super_conf_flag': 'object','Pre_harp_loan_seq_no': 'object','Prog_indicator': 'object'})
    orig_dataset_name = globals()["orig_df_"+year].compute()
    orig_dataset_name['filename'] = os.path.basename(orig_file_name)
    print(year,"--", "Obs --", len(orig_dataset_name), "Cols --", len(orig_dataset_name.columns))
    print("Read txt file with orig dask: ")

    #year=''.join(map(str, [i]))
    svcg_file_name="\sample_svcg_"+year
    globals()["svcg_df_"+year] = dd.read_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData" + svcg_file_name + ".txt",delimiter = "|",names= svcg_colnames,header = None,dtype={'Loan_seq_no':'object','Loan_age': 'float64','Curr_act_UPB': 'object','Delinq_due_disaster': 'object','Repurchase_flag': 'object','Step_modify_flag': 'object','Est_LTV': 'object','Def_payment_plan': 'object','Modify_cost':'object','Act_loss_cal': 'object','Delinq_acc_interest': 'object','Monthly_report_period': 'object','Remain_month_legal_maturity': 'object','Zero_bal_rem_UPB': 'object','Service_name': 'object','Curr_loan_delinq_status': 'object','Modify_flag': 'object'})    
    svcg_dataset_name = globals()["svcg_df_"+year].compute()
    print(year,"--", "Obs --", len(svcg_dataset_name), "Cols --", len(svcg_dataset_name.columns))
    end = time.time()
    print("Read txt file with svcg dask: ")
    #Join
    df_merge = pd.merge(orig_dataset_name, svcg_dataset_name, how="inner", on=["Loan_seq_no"])
    #Append
    master_df = master_df.append(df_merge)
    print(len(master_df),len(master_df.columns))
    del orig_dataset_name
    del svcg_dataset_name
    print("Memory released")
print("Read all files: ")
del df_merge
print("Final Memory released")

2015 -- Obs -- 50000 Cols -- 32
Read txt file with orig dask: 
2015 -- Obs -- 2707670 Cols -- 32
Read txt file with svcg dask: 
2707670 63
Memory released
2016 -- Obs -- 50000 Cols -- 32
Read txt file with orig dask: 
2016 -- Obs -- 2479776 Cols -- 32
Read txt file with svcg dask: 
5187446 63
Memory released
2017 -- Obs -- 50000 Cols -- 32
Read txt file with orig dask: 
2017 -- Obs -- 1992965 Cols -- 32
Read txt file with svcg dask: 
7180411 63
Memory released
Read all files: 
Final Memory released
Wall time: 3min 19s


In [4]:
%%time
#Formatting the dates
master_df['First_pay_dt']=pd.to_datetime(master_df['First_pay_dt'],format="%Y%m")
master_df['Maturity_dt']=pd.to_datetime(master_df['Maturity_dt'],format="%Y%m")
master_df['Monthly_report_period']=pd.to_datetime(master_df['Monthly_report_period'],format="%Y%m")
master_df['DATE']=master_df['Monthly_report_period'].dt.to_period('M')

Wall time: 2min 34s


In [5]:
%%time
#Creating the default variable by cleaning the data
master_df=master_df[master_df['Curr_loan_delinq_status']!='RA'].reset_index(drop=True)
master_df['Curr_loan_delinq_status']=master_df['Curr_loan_delinq_status'].astype('int')

Wall time: 1min 44s


In [6]:
#Count the number of unique values in "Curr_loan_delinq_status" column
master_df['Curr_loan_delinq_status'].value_counts().index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 30, 33, 32,
            34, 35, 36, 37, 39, 38, 40, 41, 42, 45, 44, 43, 46, 48, 49, 50, 47,
            51, 52, 54, 53, 55, 56, 59, 61, 60, 62, 58, 57, 63],
           dtype='int64')

In [7]:
%%time
#Create a new column and update the values based on values of "Curr_loan_delinq_status" column values
master_df['Default']=np.where(master_df['Curr_loan_delinq_status'].isin([0,1,2,3,4,5]),0,1)

Wall time: 175 ms


In [8]:
%%time
#Group all the rows with same loan sequence number and find the max_deliq vaule i.e max "Curr_loan_delinq_status" value
max_deliq=master_df.groupby('Loan_seq_no')['Curr_loan_delinq_status'].max().reset_index()
max_deliq.rename(columns={'Curr_loan_delinq_status':'Max_deliq'},inplace=True)

Wall time: 1.29 s


In [9]:
%%time
#Merge max_deliq column with master_df 
final_df=master_df.merge(max_deliq,on="Loan_seq_no")

Wall time: 2min 13s


## Reading MEV files

In [10]:
%%time
# Reading the MEV data
#Read csv files of HPI index and Unemployment rate


hpi = pd.read_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData\CSUSHPINSA.csv", parse_dates=['DATE'])
unrate = pd.read_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData\UNRATE.csv", parse_dates=['DATE'])

Wall time: 2.09 s


In [11]:
%%time
#Formatting the dates
unrate['DATE'] = unrate['DATE'].dt.to_period('M')
hpi['DATE'] = hpi['DATE'].dt.to_period('M')

Wall time: 51.1 ms


In [12]:
# change HPI MOM
hpi['HPI_Change_MOM'] = hpi['CSUSHPINSA'] - hpi['CSUSHPINSA'].shift(1)



# Relative change HPI MOM
hpi['HPI_RelChange_MOM'] = hpi['HPI_Change_MOM']/ hpi['CSUSHPINSA'].shift(1)



# change Unrate MOM
unrate['UNRATE_Change_MOM'] = unrate['UNRATE'] - unrate['UNRATE'].shift(1)



# Relative change HPI MOM
unrate['UNRATE_RelChange_MOM'] = unrate['UNRATE_Change_MOM']/ unrate['UNRATE'].shift(1)



# JOin both
MEV_df = pd.merge(unrate, hpi, how="inner", on=["DATE"])
MEV_df.head()

,DATE,UNRATE,UNRATE_Change_MOM,UNRATE_RelChange_MOM,CSUSHPINSA,HPI_Change_MOM,HPI_RelChange_MOM
0,2015-01,5.7,NaN,NaN,166.266,NaN,NaN
1,2015-02,5.5,-0.2,-0.035088,166.651,0.385,0.002316
2,2015-03,5.4,-0.1,-0.018182,168.107,1.456,0.008737
3,2015-04,5.4,0.0,0.000000,169.982,1.875,0.011154
4,2015-05,5.6,0.2,0.037037,171.860,1.878,0.011048


In [14]:
#Export MEV data
MEV_df.to_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData\MEV_df.csv", index = False)

In [15]:
%%time
# join
final_df = pd.merge(final_df, MEV_df, how="left", on=["DATE"])

Wall time: 4min 9s


In [16]:
# Check columns
final_df.columns, print("Number of Cols --", len(final_df.columns), "Number of Obs - ",len(final_df))

Number of Cols -- 72 Number of Obs -  7179805


(Index(['Credit_score', 'First_pay_dt', 'First_time_HB_Flag', 'Maturity_dt',
        'MSA', 'Mortgage_Ins%', 'No_of_units', 'Occupancy_status', 'Orig_CLTV',
        'Orig_DTI_ratio', 'Orig_UPB', 'Orig_LTV', 'Orig_Interest_rate',
        'Channel', 'PPM_flag', 'Prod_type', 'Property_state', 'Property_type',
        'Postal_code', 'Loan_seq_no', 'loan_purpose', 'Orig_loan_term',
        'No_of_borrowers', 'Seller_name', 'Service_name', 'Super_conf_flag',
        'Pre_harp_loan_seq_no', 'Prog_indicator', 'Harp_indicator',
        'Prop_val_method', 'Interest_only_Indicator', 'filename',
        'Monthly_report_period', 'Curr_act_UPB', 'Curr_loan_delinq_status',
        'Loan_age', 'Remain_month_legal_maturity', 'Repurchase_flag',
        'Modify_flag', 'Zero_bal_code', 'Zero_bal_effective_dt',
        'Curr_interest_rate', 'Curr_def_UPB', 'Due_dt_of_LPI', 'MI_recov',
        'Net_sales_proceeds', 'Non_MI_recov', 'Expenses', 'Legal_costs',
        'Maintenance_preserv_costs', 'Tax_&_Ins', 

In [17]:
#Checking the final_df file
final_df.head(50)

,Credit_score,First_pay_dt,First_time_HB_Flag,Maturity_dt,MSA,Mortgage_Ins%,No_of_units,Occupancy_status,Orig_CLTV,Orig_DTI_ratio,...,Interest_Bearing_UPB,DATE,Default,Max_deliq,UNRATE,UNRATE_Change_MOM,UNRATE_RelChange_MOM,CSUSHPINSA,HPI_Change_MOM,HPI_RelChange_MOM
0,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,417000.00,2015-03,0,0,5.4,-0.1,-0.018182,168.107,1.456,0.008737
1,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,415000.00,2015-04,0,0,5.4,0.0,0.000000,169.982,1.875,0.011154
2,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,413000.00,2015-05,0,0,5.6,0.2,0.037037,171.860,1.878,0.011048
3,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,409000.00,2015-06,0,0,5.3,-0.3,-0.053571,173.460,1.600,0.009310
4,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,409000.00,2015-07,0,0,5.2,-0.1,-0.018868,174.506,1.046,0.006030
5,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,407000.00,2015-08,0,0,5.1,-0.1,-0.019231,174.950,0.444,0.002544
6,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,405000.00,2015-09,0,0,5.0,-0.1,-0.019608,175.060,0.110,0.000629
7,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,401173.07,2015-10,0,0,5.0,0.0,0.000000,175.067,0.007,0.000040
8,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,399228.34,2015-11,0,0,5.1,0.1,0.020000,175.163,0.096,0.000548
9,734,2015-04-01,N,2030-03-01,NaN,0,1,P,80,15,...,397279.56,2015-12,0,0,5.0,-0.1,-0.019608,175.139,-0.024,-0.000137


In [18]:
%%time
#Export final df file to csv file for futher analysis
final_df.to_csv(r"D:\A complete Resuorce\DataScience\TCS FILES\LoanDataset\Documents\InitialData\final_df.csv")

Wall time: 18min 23s
